
<h2 id="data">Extracting and Visualizing Stock Data</h2>

Description

Extracting essential data from a dataset and displaying it is a necessary part of data science; therefore individuals can make correct decisions based on the data. In this assignment, you will extract some stock data, you will then display this data in a graph.



<h2 id="data">You need the following libraries for this project</h2>

In [ ]:
import pandas as pd
import yfinance as yf
import numpy as np
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
%matplotlib inline

<h2 id="data">Question 1: Use yfinance to Extract Stock Data</h2>


In [ ]:
tesla = yf.Ticker("TSLA")

In [ ]:
tesla_data = tesla.history(period="max")

In [ ]:
tesla_data.reset_index(inplace=True)
tesla_data.head()

In [ ]:
Date 	Open 	High 	Low 	Close 	Volume 	Dividends 	Stock Splits

0 	2010-06-29 	3.800 	5.000 	3.508 	4.778 	93831500 	0 	0.0
1 	2010-06-30 	5.158 	6.084 	4.660 	4.766 	85935500 	0 	0.0
2 	2010-07-01 	5.000 	5.184 	4.054 	4.392 	41094000 	0 	0.0
3 	2010-07-02 	4.600 	4.620 	3.742 	3.840 	25699000 	0 	0.0
4 	2010-07-06 	4.000 	4.000 	3.166 	3.222 	34334500 	0 	0.0

<h2 id="data">Question 2: Use Webscraping to Extract Tesla Revenue Data</h2>




Use the requests library to download the webpage url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue". Save the text of the response as a variable named html_data.



In [ ]:
url= "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"
html_data = requests.get(url).text

In [ ]:
soup = BeautifulSoup(html_data, "html.parser")
soup.find_all('title')

In [ ]:
[<title>Tesla Revenue 2010-2023 | TSLA | MacroTrends</title>]

In [ ]:
tesla_revenue = pd.DataFrame(columns = ['Date', 'Revenue'])

for row in soup.find_all("tbody")[1].find_all("tr"):
    col = row.find_all("td")
    date = col[0].text
    revenue = col[1].text.replace("$", "").replace(",", "")
    
    tesla_revenue = tesla_revenue.append({"Date": date, "Revenue": revenue}, ignore_index = True)

In [ ]:
tesla_revenue.dropna(inplace=True)
tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]


In [ ]:
tesla_revenue.tail()



In [ ]:
 	Date 	Revenue
46 	2010-09-30 	31
47 	2010-06-30 	28
48 	2010-03-31 	21
50 	2009-09-30 	46
51 	2009-06-30 	27

<h2 id="data"> Question 3: Use yfinance to Extract GME Stock Data</h2>

In [ ]:
gamestop = yf.Ticker("GME")

In [ ]:
gamestop_data = gamestop.history(period="max")

In [ ]:
gamestop_data.reset_index(inplace=True)
gamestop_data.head()

In [ ]:
 	Date 	Open 	High 	Low 	Close 	Volume 	Dividends 	Stock Splits
0 	2002-02-13 	6.480514 	6.773400 	6.413183 	6.766666 	19054000 	0.0 	0.0
1 	2002-02-14 	6.850829 	6.864295 	6.682504 	6.733002 	2755400 	0.0 	0.0
2 	2002-02-15 	6.733001 	6.749833 	6.632006 	6.699336 	2097400 	0.0 	0.0
3 	2002-02-19 	6.665671 	6.665671 	6.312189 	6.430017 	1852600 	0.0 	0.0
4 	2002-02-20 	6.463681 	6.648838 	6.413183 	6.648838 	1723200 	0.0 	0.0

<h2 id="data">Question 4: Use Webscraping to Extract GME Revenue Data</h2>

In [11]:
url1 = "https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue"

In [ ]:
html_data = requests.get(url1).text

In [ ]:
soup = BeautifulSoup(html_data, "html.parser")
soup.find_all('title')

[<title>GameStop Revenue 2010-2023 | GME | MacroTrends</title>]

In [ ]:
gme_revenue = pd.DataFrame(columns=['Date', 'Revenue'])

for table in soup.find_all('table'):

    if ('GameStop Quarterly Revenue' in table.find('th').text):
        rows = table.find_all('tr')
        
        for row in rows:
            col = row.find_all('td')
            
            if col != []:
                date = col[0].text
                revenue = col[1].text.replace(',','').replace('$','')

                gme_revenue = gme_revenue.append({"Date":date, "Revenue":revenue}, ignore_index=True)

In [ ]:
gme_revenue.tail()

In [ ]:
 	Date 	Revenue
49 	2010-01-31 	3524
50 	2009-10-31 	1835
51 	2009-07-31 	1739
52 	2009-04-30 	1981
53 	2009-01-31 	3492

<h2 id="data">Question 5: Plot Tesla Stock Graph</h2>

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data.Date, infer_datetime_format=True), y=stock_data.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data.Date, infer_datetime_format=True), y=revenue_data.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

In [ ]:
make_graph(tesla_data, tesla_revenue, 'Tesla')

<h2 id="data">Question 6: Plot GameStop Stock Graph</h2>

In [ ]:
make_graph(gamestop_data, gme_revenue, 'GameStop')

In [ ]:
print("Completed")